# QA on single documents drawn from a corpus of individual files.

> Building a simple Gradio application to allow the user to ask questions about a single file. 

In [ ]:
#| default_exp tools.single_document_qa

In [1]:
#| export

import os
import pandas as pd

from alhazen.core import TaskInstructionRegistry, get_langchain_llm, get_langchain_embeddings, GGUF_LOOKUP_URL, MODEL_TYPE
import alhazen.utils.nxml_text_extractor as te 
from alhazen.utils.pdf_research_article_text_extractor import PyMuPDFBlockLoader, PyMuPDFBlockParser, PyMuPDFBlock

import re

from InstructorEmbedding import INSTRUCTOR 

from pathlib import Path

from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import AnalyzeDocumentChain,  RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import NLTKTextSplitter, CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

import gradio as gr

import fire

[nltk_data] Downloading package punkt to /Users/gburns/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [6]:
#| export 

PROMPT_SINGLE_ARTICLE_QUESTION = {
    'name': 'SingleArticleQuestion',
    'description': 'Ask a question about a single research paper',
    'system':   
        '''You are an helpful assistant, well-versed in scientific language, but cautious about making sure what you say is accurate. 
        Your task is to answer a question about a progress report concisely and accurately.
        Answer the question briefly in a single paragraph or bullet-point list (if needed). 
        The question will be enclosed by angle brackets. 
        The text of the progress report will be enclosed by square brackets.
        Remember - answer the with the minimum of commentary possible in a single paragraph or list.''',
    'instruction': 
        '''First read the text of this progress report (enclosed in double square brackets):
        ARTICLE: [[{article}]]
        Then, take a deep breath and think carefully before responding as precisely as possible to this question (enclosed in double angle brackets):
        QUESTION: <<{question}>>''',
    'input_variables': ["article", "question"]}

class FileCollectionLangChainChatBot:
    '''Uses a simple prompt to answer questions about a collection of short PDF files.'''

    def __init__(self, 
                doc_dir, 
                llm_name,
                embed_model_name='hkunlp/instructor-large',
                chunk_size=1000,
                chunk_overlap=100):
        
        self.doc_dir = doc_dir
        if doc_dir[-1:] != '/':
            doc_dir += '/'
        self.change_directory(self.doc_dir)
        self.prompt_registry = TaskInstructionRegistry()
        self.prompt_registry.register_new_instruction_template(PROMPT_SINGLE_ARTICLE_QUESTION)
        model_type = None

        if llm_name == 'gpt-4':
            model_type = MODEL_TYPE.OpenAI
        elif GGUF_LOOKUP_URL.get(llm_name) is not None:
            model_type = MODEL_TYPE.LlamaCpp
        
        t = self.prompt_registry.get_instruction_template(PROMPT_SINGLE_ARTICLE_QUESTION.get('name')) 
        if model_type == MODEL_TYPE.OpenAI:
            self.prompt_template = t.generate_prompt_template()
        else:
            self.prompt_template = t.generate_llama2_promp_templatet()

        self.llm = get_langchain_llm(llm_name)   
        self.qa_chain = load_qa_chain(self.llm, chain_type="map_reduce") 

        #self.text_splitter = RecursiveCharacterTextSplitter(
        #    # Set a really small chunk size, just to show.
        #    chunk_size = chunk_size,
        #    chunk_overlap  = chunk_overlap,
        #    length_function = len,
        #    is_separator_regex = False,
        #)
        self.text_splitter = NLTKTextSplitter(chunk_size=chunk_size, 
                                         chunk_overlap=chunk_overlap)

        self.embed_model = HuggingFaceInstructEmbeddings(model_name=embed_model_name,
                                                         model_kwargs={'device': 'mps'})
        self.vectorstore = Chroma("langchain_store", self.embed_model)

        self.qa_chain = RetrievalQA.from_chain_type(llm=self.llm,
                                                    chain_type="stuff", 
                                                    retriever=self.vectorstore.as_retriever())

    def load_one_pdf_document(self, file_path):
        pdf_loader = PyMuPDFBlockLoader(file_path)
        docs = pdf_loader.load()
        #html_paragraphs = '<p>'+'</p><p>'.join([d.page_content for d in docs])+'</p>'
        #html_sentences = self.sent_detector.tokenize(html_paragraphs.strip())
        # = '<br>'.join(html_sentences)

        split_docs = self.text_splitter.split_documents(docs)
        self.vectorstore.delete_collection
        self.vectorstore.add_documents(split_docs)

        html_text = '<p>'+re.sub('\n\n', '</p><p>', docs[0].page_content)+'</p>'
        html_text = re.sub('\n', '<br>', html_text)
        return html_text
        
    def run_batch_of_questions_over_each_file(self, questions):
        l =[]
        # List all pdf files in doc_dir by walking the whole directory tree
        pdf_file_list = []
        for root, dirs, files in os.walk(self.doc_dir):
            for file in files:
                print(file)
                if file.endswith('.pdf'):
                    loader = PyMuPDFLoader(os.path.join(root, file))
                    docs = loader.load()
                    report_text = '\n'.join([d.page_content for d in docs])
                    for q in questions:
                        p = self.prompt_template.format(article=report_text, question=q)
                        try:
                            a = self.llm(p)
                            print('\t\t%s'%(a))
                            l.append({'file': file, 'question': q, 'answer': a})
                        except Exception as e: 
                            print('ERROR ASKING "%s" on "%s"'%(q, file)) 
                            print(e) 
        df = pd.DataFrame(l)
        return df    

    def change_directory(self, doc_dir):
        if doc_dir[-1:] != '/':
            doc_dir += '/'
        file_list = []
        dir_list = []
        with os.scandir(doc_dir) as it:
            for entry in it:
                suffix = Path(entry.path).suffix
                if entry.is_file():
                    if suffix in ['.pdf', '.PDF', '.txt', '.TXT', '.xml', '.XML', '.nxml', '.NXML']:
                        file_list.append(entry.name)
                elif entry.is_dir():
                    dir_list.append(entry.name)
        file_list.sort()
        dir_list.sort()

        # insert '..' into dir_list
        dir_list.insert(0, '..')

        self.doc_dir = doc_dir
        self.folder_df = pd.DataFrame(dir_list, columns=['folder'])
        self.file_df = pd.DataFrame(file_list, columns=['file'])
        return (doc_dir, self.folder_df, self.file_df)

    def run_gradio(self):

        '''def load_jats_paper(doi):
            df = self.get_ft_url_from_doi(doi)
            print(df.columns)
            if df.shape[0] == 0:
                return('No paper found with that DOI')
            title = df['title'].values[0]
            first_author = df['last_name'].values[0]
            
            year = df['year'].values[0]  
            url = df['full_text_url'].values[0]
            xml = requests.get(url).text
            doc = te.NxmlDoc(doi, xml)
            df1 = doc.build_simple_document_dataframe()
            df_text = df1[df1['FIG_REF'] == '']
            for i, row in df_text.iterrows():
                if row.TAG != 'p' and row.PLAIN_TEXT[-1:]!='.':
                    row.PLAIN_TEXT += '.'
            text_list = df_text['PLAIN_TEXT'].values.tolist()
            text_length = sum([len(t) for t in text_list])
            documents = [Document(text=t) for t in text_list]
            self.index = VectorStoreIndex.from_documents(documents, service_context=self.service_context)
            self.query_engine = self.index.as_query_engine()
            return('Loaded paper: ' + first_author + ' (' + str(year) + ') "' + title + '\n' + str(text_length))'''

        def add_text(history, text):
            #print('add_text: history: %s, text: %s'%(history, text))
            history = history + [(text, None)]
            return history, gr.Textbox(value="", interactive=False)

        def select_dir(evt: gr.SelectData):
            new_dir = self.doc_dir + '/' + evt.value      
            if evt.value == '..':
                new_dir = str(Path(self.doc_dir).parent)
            print('select_dir: %s'%(new_dir))
            return self.change_directory(new_dir)

            #doc_text = f"You selected {evt.value} at {evt.index} from {evt.target}"
            #return doc_text
        
        def select_file(evt: gr.SelectData):
            file_path = self.doc_dir + '/' + evt.value  
            html_text = f"You selected {evt.value} at {evt.index} from {evt.target}"
            if file_path.endswith('.pdf'):
                html_text = self.load_one_pdf_document(file_path)
            return html_text
            
        def add_file(history, file):
            #print('add_history: history: %s, file: %s'%(history, file))
            history = history + [((file.name,), None)]
            return history

        def clear_chat(history):
            return []

        def bot(history):
            #print('bot: history: %s'%(history))
            # prompt to send to the agent is the last message from the user
            prompt = history[-1][0]
            response = self.qa_chain({"query": prompt})
            print('RESPONSE: %s'%(str(response)))
            history[-1][1] = str(response.get('result','No answer found'))
            print('WHOLE HISTORY: %s'%(history))
            return history

        with gr.Blocks() as demo:
            with gr.Tab("Corpus"):
                with gr.Row():
                    with gr.Column():
                        doc_dir = gr.Textbox(show_label=False, lines=1, value=self.doc_dir, interactive=False)
                        with gr.Row():
                            directories = gr.DataFrame(show_label=False, value=self.folder_df, interactive=False)
                            files = gr.DataFrame(show_label=False, value=self.file_df, interactive=False)
                    doc_text = gr.HTML(label="File Contents", lines=20)
                directories.select(select_dir, None, [doc_dir, directories, files], queue=False )   
                files.select(select_file, None, doc_text , queue=False )

            with gr.Tab("Chat"):
                chatbot = gr.Chatbot(
                    [],
                    elem_id="chatbot",
                    bubble_full_width=False,
                    #avatar_images=(None, files(alhazen_resources).joinpath('alhazen.png'))
                    
                )
                with gr.Row():
                    txt = gr.Textbox(
                        scale=4,
                        show_label=False,
                        placeholder="Enter text and press enter, or upload files",
                        container=False,
                    )
                    clear_btn = gr.Button("Clear")

                txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(bot, chatbot, chatbot)
                txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)                
                clear_btn.click(clear_chat, [], [chatbot], queue=False)
                
        demo.queue()
        demo.launch()

#os.environ['LLMS_TEMP_DIR'] = '/tmp/alhazen'
#cb = FileCollectionLangChainChatBot('/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/reading/cryoet_papers/', 'mistral-7b-instruct')
#cb.load_one_pdf_document('/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/reading/cryoet_papers/de_teresa-2022-10.1101-2022.04.12.488077.pdf')
#cb.run_gradio()
#cb.run_gradio()

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /tmp/alhazen/mistral-7b-instruct-v0.1.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q6_K     [ 14336,

load INSTRUCTOR_Transformer
max_seq_length  512


/var/folders/pl/z8kd7hxd3qv9_0f1_ndg3hxm0000gq/T/ipykernel_74736/2795494882.py:206: GradioUnusedKwargWarning: You have unused kwarg parameters in HTML, please remove them: {'lines': 20}
  doc_text = gr.HTML(label="File Contents", lines=20)
/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/utils.py:810: UserWarning: Expected 1 arguments for function <function FileCollectionLangChainChatBot.run_gradio.<locals>.clear_chat at 0x46843c2c0>, received 0.
  warnings.warn(
/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/utils.py:814: UserWarning: Expected at least 1 arguments for function <function FileCollectionLangChainChatBot.run_gradio.<locals>.clear_chat at 0x46843c2c0>, received 0.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [6]:
#os.environ['LLMS_TEMP_DIR'] = '/tmp/alhazen'
#cb = FileCollectionLangChainChatBot('/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/reading/cryoet_papers/', 'llama-2-70b-chat')
#cb.load_one_pdf_document('/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/reading/cryoet_papers/de_teresa-2022-10.1101-2022.04.12.488077.pdf')
#response = cb.qa_chain({"query": 'List the types of organisms or animals used in the paper (yeast, zebrafish, mice, etc)'})
#print(response)

llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from /tmp/alhazen/llama-2-70b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  8192,  8192, 

load INSTRUCTOR_Transformer
max_seq_length  512
 The paper uses yeast cells (Saccharomyces cerevisiae and Schizosaccharomyces pombe) for its experiments.{'query': 'List the types of organisms or animals used in the paper (yeast, zebrafish, mice, etc)', 'result': ' The paper uses yeast cells (Saccharomyces cerevisiae and Schizosaccharomyces pombe) for its experiments.'}



llama_print_timings:        load time = 10490.90 ms
llama_print_timings:      sample time =    51.46 ms /    32 runs   (    1.61 ms per token,   621.88 tokens per second)
llama_print_timings: prompt eval time = 32334.47 ms /  1695 tokens (   19.08 ms per token,    52.42 tokens per second)
llama_print_timings:        eval time =  5729.82 ms /    31 runs   (  184.83 ms per token,     5.41 tokens per second)
llama_print_timings:       total time = 38261.56 ms


In [7]:
#response = cb.qa_chain({"query": 'Do they use any cells from multicellular animals, birds, or fish?'})
#print(response)

Llama.generate: prefix-match hit


 Yes, they do use cells from a multicellular animal, specifically HeLa cells (human cervical cancer cells) in their experiments.{'query': 'Do they use any cells from multicellular animals, birds, or fish?', 'result': ' Yes, they do use cells from a multicellular animal, specifically HeLa cells (human cervical cancer cells) in their experiments.'}



llama_print_timings:        load time = 10490.90 ms
llama_print_timings:      sample time =    47.75 ms /    31 runs   (    1.54 ms per token,   649.16 tokens per second)
llama_print_timings: prompt eval time = 23392.13 ms /  1100 tokens (   21.27 ms per token,    47.02 tokens per second)
llama_print_timings:        eval time =  5330.84 ms /    30 runs   (  177.69 ms per token,     5.63 tokens per second)
llama_print_timings:       total time = 28908.21 ms
